In [1]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.support.ui import Select

In [2]:
#pip install selenium

In [23]:
# specify the url
urlpage = 'https://lpse.lkpp.go.id/eproc4/lelang' 
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Firefox(executable_path = 'geckodriver.exe')

https://lpse.lkpp.go.id/eproc4/lelang


In [24]:
#menyiapkan variable tabel, variabel tabel nantinya berisi tabel dalam bentuk string menggunakan bahasa HTML
tabel=''
#meminta driver firefox membuka alamat web
driver.get(urlpage)
#meminta driver mencari dan memilih drop down list
drop_down = Select(driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/div[1]/div[1]/div/label/select'))
#meminta driver memilih text berupa 100
drop_down.select_by_visible_text('100')
#meminta driver beristirahat selama 10 detik
time.sleep(10)
for itr in range(1,30):
    #meminta driver mencari element berdasarkan xpath
    elm = driver.find_element_by_xpath('//*[@id="tbllelang"]')
    #meminta driver mengambil inspect element dari browser pada saat itu (tanpa diubah oleh javascript)
    data=elm.get_attribute("innerHTML")
    #memasukkan data ke dalam tabel
    tabel=tabel+"\n\n\n\n\n\n\n\n\n\n"+str(data)
    #meminta driver mencari tombol next berdasarkan xpath
    next_button = driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/div[3]/div[2]/div/ul/li[10]/a')
    #meminta driver melakukan klik pada tombol next
    next_button.click()
    time.sleep(10)
    #lakukan perintah perulangan di atas sesuai dengan kebutuhan (in range)

driver.quit()
print(tabel)












                <thead>
                <tr role="row"><th class="sorting_desc" tabindex="0" aria-controls="tbllelang" rowspan="1" colspan="1" style="width: 33px;" aria-sort="descending" aria-label="Kode: activate to sort column ascending">Kode</th><th class="sorting" tabindex="0" aria-controls="tbllelang" rowspan="1" colspan="1" style="width: 561px;" aria-label="Nama Paket: activate to sort column ascending">Nama Paket</th><th class="sorting" tabindex="0" aria-controls="tbllelang" rowspan="1" colspan="1" style="width: 196px;" aria-label="Instansi: activate to sort column ascending">Instansi</th><th class="sorting_disabled" rowspan="1" colspan="1" style="width: 180px;" aria-label="Tahap">Tahap</th><th class="sorting" tabindex="0" aria-controls="tbllelang" rowspan="1" colspan="1" style="width: 27px;" aria-label="HPS: activate to sort column ascending">HPS</th></tr>
                </thead>
            <tbody><tr role="row" class="odd"><td class="sorting_1">6599119</td><td><p>

In [7]:
driver.quit()

Di bawah ini, cara membuat tabel HTML di atas menjadi data frame


sumber : https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [25]:
import lxml.html as lh

In [26]:
doc = lh.fromstring(tabel)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [27]:
[len(T) for T in tr_elements[:12]]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [28]:
#Create empty list
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Kode"
2:"Nama Paket"
3:"Instansi"
4:"Tahap"
5:"HPS"


In [29]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=5:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [30]:
[len(C) for (title,C) in col]

[2928, 2928, 2928, 2928, 2928]

In [31]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [32]:
df.head()

,Kode,Nama Paket,Instansi,Tahap,HPS
0,6599119,Tenaga Ahli Data Driven Kegiatan Koordinasi Se...,Kementerian Perencanaan Pembangunan Nasional,Pengumuman Pascakualifikasi [...],120 Jt
1,6598119,Tenaga Ahli IT Analisa Sistem Kegiatan Koord...,Kementerian Perencanaan Pembangunan Nasional,Pengumuman Pascakualifikasi [...],"183,5 Jt"
2,6597119,Tenaga Ahli Senior Monitoring dan Evaluasi Str...,Kementerian Perencanaan Pembangunan Nasional,Pengumuman Pascakualifikasi [...],"228,4 Jt"
3,6596119,Tenaga Ahli Senior Perencanaan Strategic Natio...,Kementerian Perencanaan Pembangunan Nasional,Pengumuman Pascakualifikasi [...],"254,1 Jt"
4,6595119,Tenaga Ahli Muda Bidang Energy Policy Kegiatan...,Kementerian Perencanaan Pembangunan Nasional,Pengumuman Pascakualifikasi [...],132 Jt


In [33]:
df.tail()

,Kode,Nama Paket,Instansi,Tahap,HPS
2923,1029119,Jasa Kebersihan di Lingkungan Sekretariat Kabi...,Sekretariat Kabinet,Tender Sudah Selesai,"783,1 Jt"
2924,1027119,"Pemeliharaan Prasarana TIK (Jaringan Komputer,...",Kementerian Perencanaan Pembangunan Nasional,Tender Sudah Selesai,"1,3 M"
2925,1026119,Pemeliharaan Sistem Transportasi Gedung spse 3...,Kementerian Perencanaan Pembangunan Nasional,Tender Sudah Selesai,"272,2 Jt"
2926,1025119,Pemeliharaan Taman spse 3 Jasa Lainnya - TA 2...,Kementerian Perencanaan Pembangunan Nasional,Tender Sudah Selesai,249 Jt
2927,1024119,Sewa Mesin Fotocopy spse 3 Jasa Lainnya - TA ...,Kementerian Perencanaan Pembangunan Nasional,Tender Sudah Selesai,"2,5 M"


In [34]:
df.to_csv('eproc.csv',index = False, mode='a')